# Informe: Exploración de Datos y Modelado Predictivo

Este informe detalla el enfoque adoptado para la exploración de datos (EDA), el preprocesamiento y el modelado predictivo utilizado para predecir temperaturas futuras basadas en datos históricos del clima. A su vez este archivo fue usado como prueba de todas las funciones que se crearon para el proyecto, todas estas funciones se probaron aqui y se adaptaron para su mejor uso y la oprimizacion del modelo para el producto final

# 1. Exploración de Datos (EDA)
## 1.1 Análisis de los Datos Iniciales

Los datos meteorológicos fueron obtenidos utilizando la API de WeatherAPI y se procesaron en formato JSON. Cada registro incluía las siguientes variables:

* Fecha: Día específico del registro.
* Temperatura promedio: (average_temp) Temperatura promedio diaria en grados Celsius.
* Temperatura máxima y mínima: (max_temp, min_temp) Temperaturas extrema del día.
* Precipitación: (precipitation) Cantidad total de lluvia en milímetros.
* Humedad: (humidity) Promedio de la humedad relativa durante el día.
* Condiciones climáticas: (condition) Descripción textual del clima.

## 1.2 Proceso de Preprocesamiento
Los datos históricos fueron convertidos a un DataFrame para facilitar su manipulación. Se aplicaron las siguientes transformaciones:

1.  Creación de características dinámicas:
* Temperatura promedio móvil: (rolling_avg_temp) Promedio de la temperatura de los últimos 3 días.
* Máxima temperatura móvil: (rolling_max_temp) Valor máximo de los últimos 3 días.
* Mínima temperatura móvil: (rolling_min_temp) Valor mínimo de los últimos 3 días.
* Precipitación acumulativa: (cumulative_precipitation) Total acumulado de precipitación hasta la fecha actual.

2. Eliminación de columnas irrelevantes:

* La columna condition fue eliminada porque frecuentemente estaba vacía o tenía valores inconsistentes.
* La columna date no fue usada directamente como característica del modelo.

3. Separación de datos:

* Características (X*): Variables predictoras que incluyen temperaturas, humedad y precipitaciones.
* Objetivo (y): La temperatura promedio (average_temp) que deseamos predecir.


## 2. Validación y Ajustes de Datos
### 2.1 Verificación de valores faltantes

Se identificaron valores faltantes en las características. Estos fueron reemplazados utilizando la mediana de la columna correspondiente para minimizar el impacto en el modelo.

### 2.2 Detección de valores extremos
Los valores extremos fueron identificados utilizando el método del rango intercuartílico (IQR). Se aplicó truncado para reducir el impacto de estos valores en el modelo.

### 2.3 Análisis de correlaciones
Se analizaron las correlaciones entre las características y el objetivo para identificar las más relevantes. Las características con correlación absoluta menor a 0.1 fueron eliminadas por considerarse no significativas para el modelo.

In [1]:
from model import preprocess_weather_data, validate_and_adjust_data

# Simulamos datos preprocesados 60 días de datos meteorológicos para simular lo que se obtendría de una API
historical_data = [
    {"date": "2024-01-01", "average_temp": 2, "max_temp": 25, "min_temp": 15, "precipitation": 2, "humidity": 80},
    {"date": "2024-01-02", "average_temp": 22, "max_temp": 27, "min_temp": 17, "precipitation": 0, "humidity": 75},
    {"date": "2024-01-03", "average_temp": 21, "max_temp": 26, "min_temp": 16, "precipitation": 1, "humidity": 78},
    {"date": "2024-01-04", "average_temp": 23, "max_temp": 28, "min_temp": 18, "precipitation": 0, "humidity": 72},
    {"date": "2024-01-05", "average_temp": 24, "max_temp": 29, "min_temp": 19, "precipitation": 0, "humidity": 70},
    {"date": "2024-01-06", "average_temp": 25, "max_temp": 30, "min_temp": 20, "precipitation": 0, "humidity": 68},
    {"date": "2024-01-07", "average_temp": 26, "max_temp": 31, "min_temp": 21, "precipitation": 0, "humidity": 65},
    {"date": "2024-01-08", "average_temp": 18, "max_temp": 26, "min_temp": 17, "precipitation": 4, "humidity": 53},
    {"date": "2024-01-09", "average_temp": 19, "max_temp": 27, "min_temp": 18, "precipitation": 3, "humidity": 55},
    {"date": "2024-01-10", "average_temp": 20, "max_temp": 28, "min_temp": 19, "precipitation": 2, "humidity": 58},
    {"date": "2024-01-11", "average_temp": 28, "max_temp": 30, "min_temp": 25, "precipitation": 1, "humidity": 60},
    {"date": "2024-01-12", "average_temp": 29, "max_temp": 31, "min_temp": 26, "precipitation": 0, "humidity": 62},
    {"date": "2024-01-13", "average_temp": 30, "max_temp": 32, "min_temp": 27, "precipitation": 0, "humidity": 64},
    {"date": "2024-01-14", "average_temp": 31, "max_temp": 33, "min_temp": 28, "precipitation": 0, "humidity": 66},
    {"date": "2024-01-15", "average_temp": 1, "max_temp": 34, "min_temp": 29, "precipitation": 0, "humidity": 68},
    {"date": "2024-01-16", "average_temp": 33, "max_temp": 35, "min_temp": 30, "precipitation": 0, "humidity": 70},
    {"date": "2024-01-17", "average_temp": 34, "max_temp": 36, "min_temp": 31, "precipitation": 0, "humidity": 72},
    {"date": "2024-01-18", "average_temp": 35, "max_temp": 37, "min_temp": 32, "precipitation": 0, "humidity": 74},
    {"date": "2024-01-19", "average_temp": 36, "max_temp": 38, "min_temp": 33, "precipitation": 0, "humidity": 76},
    {"date": "2024-01-20", "average_temp": 37, "max_temp": 39, "min_temp": 34, "precipitation": 0, "humidity": 78},
    {"date": "2024-01-21", "average_temp": 38, "max_temp": 40, "min_temp": 35, "precipitation": 0, "humidity": 80},
    {"date": "2024-01-22", "average_temp": 39, "max_temp": 41, "min_temp": 36, "precipitation": 0, "humidity": 82},
    {"date": "2024-01-23", "average_temp": 40, "max_temp": 42, "min_temp": 37, "precipitation": 0, "humidity": 84},
    {"date": "2024-01-24", "average_temp": 41, "max_temp": 43, "min_temp": 38, "precipitation": 0, "humidity": 86},
    {"date": "2024-01-25", "average_temp": 42, "max_temp": 44, "min_temp": 39, "precipitation": 0, "humidity": 88},
    {"date": "2024-01-26", "average_temp": 43, "max_temp": 45, "min_temp": 40, "precipitation": 0, "humidity": 90},
    {"date": "2024-01-27", "average_temp": 44, "max_temp": 46, "min_temp": 41, "precipitation": 0, "humidity": 92},
    {"date": "2024-01-28", "average_temp": 45, "max_temp": 47, "min_temp": 42, "precipitation": 0, "humidity": 94},
    {"date": "2024-01-29", "average_temp": 46, "max_temp": 48, "min_temp": 43, "precipitation": 0, "humidity": 96},
    {"date": "2024-01-30", "average_temp": 47, "max_temp": 49, "min_temp": 44, "precipitation": 0, "humidity": 98},
    {"date": "2024-01-31", "average_temp": 7, "max_temp": 50, "min_temp": 45, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-01", "average_temp": 8, "max_temp": 51, "min_temp": 46, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-02", "average_temp": 9, "max_temp": 52, "min_temp": 47, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-03", "average_temp": 10, "max_temp": 53, "min_temp": 48, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-04", "average_temp": 11, "max_temp": 54, "min_temp": 49, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-05", "average_temp": 12, "max_temp": 55, "min_temp": 50, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-06", "average_temp": 13, "max_temp": 56, "min_temp": 51, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-07", "average_temp": 14, "max_temp": 57, "min_temp": 52, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-08", "average_temp": 15, "max_temp": 58, "min_temp": 53, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-09", "average_temp": 16, "max_temp": 59, "min_temp": 54, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-10", "average_temp": 17, "max_temp": 60, "min_temp": 55, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-11", "average_temp": 18, "max_temp": 61, "min_temp": 56, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-12", "average_temp": 19, "max_temp": 62, "min_temp": 57, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-13", "average_temp": 20, "max_temp": 63, "min_temp": 58, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-14", "average_temp": 21, "max_temp": 64, "min_temp": 59, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-15", "average_temp": 22, "max_temp": 65, "min_temp": 60, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-16", "average_temp": 23, "max_temp": 66, "min_temp": 61, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-17", "average_temp": 24, "max_temp": 67, "min_temp": 62, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-18", "average_temp": 25, "max_temp": 68, "min_temp": 63, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-19", "average_temp": 26, "max_temp": 69, "min_temp": 64, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-20", "average_temp": 27, "max_temp": 70, "min_temp": 65, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-21", "average_temp": 28, "max_temp": 71, "min_temp": 66, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-22", "average_temp": 29, "max_temp": 72, "min_temp": 67, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-23", "average_temp": 30, "max_temp": 73, "min_temp": 68, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-24", "average_temp": 31, "max_temp": 74, "min_temp": 69, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-25", "average_temp": 32, "max_temp": 75, "min_temp": 70, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-26", "average_temp": 33, "max_temp": 76, "min_temp": 71, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-27", "average_temp": 34, "max_temp": 77, "min_temp": 72, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-28", "average_temp": 35, "max_temp": 78, "min_temp": 73, "precipitation": 0, "humidity": 100},
    {"date": "2024-02-29", "average_temp": 36, "max_temp": 79, "min_temp": 74, "precipitation": 0, "humidity": 100}
]

# Probanos la función de preprocesamiento
X, y = preprocess_weather_data(historical_data)

# Probando la función de validación y ajuste de datos
X_adjusted, y_adjusted = validate_and_adjust_data(X, y)

# Imprimimos los datos preprocesados
print("\nDatos ajustados:")
print(X_adjusted.head())
print("\nObjetivo ajustado:")
print(y_adjusted.head())

=== INICIO: Validación de los datos preprocesados ===
Valores faltantes totales: 0
Resumen de características:
        max_temp  min_temp  precipitation    humidity  rolling_avg_temp  \
count  60.000000  60.00000      60.000000   60.000000         60.000000   
mean   50.033333  44.25000       0.216667   87.566667         25.561111   
std    16.687524  17.80842       0.738566   15.207454         10.333150   
min    25.000000  15.00000       0.000000   53.000000          2.000000   
25%    34.750000  29.75000       0.000000   74.750000         19.000000   
50%    49.500000  44.50000       0.000000   99.000000         24.000000   
75%    64.250000  59.25000       0.000000  100.000000         33.500000   
max    79.000000  74.00000       4.000000  100.000000         46.000000   

       rolling_max_temp  rolling_min_temp  cumulative_precipitation  
count         60.000000         60.000000                 60.000000  
mean          50.200000         42.166667                 11.633333  
std

### 3.2 Proceso de entrenamiento
1. División de datos:

* El conjunto de datos fue dividido en entrenamiento (80%) y prueba (20%) utilizando train_test_split.

2. Entrenamiento:

* Se entrenó el modelo RandomForestRegressor con 100 árboles y una semilla fija para reproducibilidad.

3. Evaluación:

* Se utilizaron métricas comunes como:
    * Mean Squared Error (MSE): Error cuadrático medio.
    * Mean Absolute Error (MAE): Error absoluto medio.
    * R² Score: Coeficiente de determinación que mide la varianza explicada por el modelo.
* Resultados del modelo:
    * MSE: Bajo, indicando predicciones cercanas a los valores reales.
    * MAE: Bajo, confirmando una precisión alta en las predicciones.
    * R² Score: Alto (cercano a 1), mostrando que el modelo explica la mayoría de la variabilidad en los datos.

## 4. Predicción de Clima Futuro
Se creó una función para predecir el clima de los próximos días:

1.  Entrada:
* Últimos datos conocidos del clima (last_known_data).
2. Simulación:
* Se inicializaron las características requeridas y se simularon condiciones futuras.
* Se introdujo una ligera variación aleatoria para reflejar la incertidumbre en las predicciones climáticas.
3. Salida:
* Lista de predicciones con fecha y temperatura promedio estimada.


In [2]:
from model import create_and_train_model

# Probando la función de creación y entrenamiento del modelo

model, predictions = create_and_train_model(historical_data)

=== INICIO: Validación de los datos preprocesados ===
Valores faltantes totales: 0
Resumen de características:
        max_temp  min_temp  precipitation    humidity  rolling_avg_temp  \
count  60.000000  60.00000      60.000000   60.000000         60.000000   
mean   50.033333  44.25000       0.216667   87.566667         25.561111   
std    16.687524  17.80842       0.738566   15.207454         10.333150   
min    25.000000  15.00000       0.000000   53.000000          2.000000   
25%    34.750000  29.75000       0.000000   74.750000         19.000000   
50%    49.500000  44.50000       0.000000   99.000000         24.000000   
75%    64.250000  59.25000       0.000000  100.000000         33.500000   
max    79.000000  74.00000       4.000000  100.000000         46.000000   

       rolling_max_temp  rolling_min_temp  cumulative_precipitation  
count         60.000000         60.000000                 60.000000  
mean          50.200000         42.166667                 11.633333  
std

In [5]:
# Corroborando que el modelo se haya entrenado correctamente checando las características esperadas
expected_features = model.feature_names_in_
print(expected_features)

['max_temp' 'min_temp' 'precipitation' 'humidity' 'rolling_avg_temp'
 'rolling_max_temp' 'rolling_min_temp' 'cumulative_precipitation']


In [4]:
from model import predict_future_weather

# Simulamos datos actuales para predecir el clima futuro
current_weather = {'date': '2024-12-25', 'average_temp': 8.8, 'max_temp': 10.4, 'min_temp': 7.3, 'precipitation': 0.0, 'humidity': 92, 'condition': None}


# Probamos la función de predicción de clima futuro
predictions = predict_future_weather(model, current_weather, num_days=10)
print("Predicciones futuras:")
print(predictions)


Predicciones futuras:
[14.56, 15.02, 17.92, 18.28, 18.96, 18.98, 18.99, 19.11, 19.11, 19.12]


## 5. Visualización
### 5.1 Gráfico de Tendencias Climáticas
Por mas que esto no se probo y no se encuentra aqui, el grafico se ve en el producto final y se puede ver la funcion en el mismo archivo que el resto 'model.py'
Se generó un gráfico que combina:

* Datos históricos: Mostrados como una línea sólida azul.
* Predicciones futuras: Mostradas como una línea discontinua naranja.

Este gráfico permite comparar visualmente las tendencias pasadas y futuras, ayudando en la interpretación de las predicciones.

## 6. Conclusión
El proceso implementado asegura que los datos meteorológicos sean validados y preprocesados para obtener predicciones confiables. El uso de RandomForestRegressor permitió capturar relaciones no lineales, obteniendo un modelo robusto y preciso. Las predicciones generadas ofrecen una herramienta útil para analizar tendencias climáticas futuras.

Si deseas más detalles o el código fuente, consulta el repositorio del proyecto en GitHub.